# Flight Data Clean Up

## Load source data into a single dataframe to explore

In [1]:
# Import Dependencies
import pandas as pd
from pathlib import Path
import glob
import zipfile
import os
import shutil

In [2]:
# List of all source zips
zips = ['Resources/BTS/BTS_Q1_2023.zip','Resources/BTS/BTS_Q2_2023.zip',
       'Resources/BTS/BTS_Q3_2023.zip','Resources/BTS/BTS_Q4_2023.zip']

# Directory to extract to, confirm it exists
extraction_path = 'Resources/BTS/extracted_zips'
os.makedirs(extraction_path, exist_ok=True)

# Iterate through zips and extract all
for zip in zips:
    with zipfile.ZipFile(zip, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)

    print(f"Extracted {zip} to {extraction_path}")

Extracted Resources/BTS/BTS_Q1_2023.zip to Resources/BTS/extracted_zips
Extracted Resources/BTS/BTS_Q2_2023.zip to Resources/BTS/extracted_zips
Extracted Resources/BTS/BTS_Q3_2023.zip to Resources/BTS/extracted_zips
Extracted Resources/BTS/BTS_Q4_2023.zip to Resources/BTS/extracted_zips


In [3]:
# Read all extracted CSV's
# glob is an efficient way to work through all CSV's in a directory: https://docs.python.org/3/library/glob.html
csv_files = glob.glob(f"{extraction_path}/*.csv")

# Empty list for storing DataFrames
dataframes = []

# Iterate through csv's and store into DataFrames list
for csv_file in csv_files:
    df = pd.read_csv(csv_file, low_memory=False)
    dataframes.append(df)
# Concatenante into single DataFrame
unfiltered_flight_info = pd.concat(dataframes, ignore_index = True)
unfiltered_flight_info.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,...,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
0,2023,1,1,22,7,2023-01-22,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,2023,1,1,22,7,2023-01-22,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,2023,1,1,22,7,2023-01-22,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
3,2023,1,1,22,7,2023-01-22,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
4,2023,1,1,22,7,2023-01-22,B6,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [4]:
# Specify columns and remove limitation on display
pd.set_option('display.max_columns', None)
flights_df = pd.DataFrame(unfiltered_flight_info[['FlightDate','IATA_Code_Marketing_Airline', 
                                                  'Operated_or_Branded_Code_Share_Partners',
                                                  'Operating_Airline ','Flight_Number_Marketing_Airline',
                                                  'Origin','Dest','DepDelay','DepDelayMinutes',
                                                  'DepDel15','ArrDelay','ArrDelayMinutes','Cancelled',
                                                  'Diverted','CarrierDelay','WeatherDelay','NASDelay',
                                                  'SecurityDelay','LateAircraftDelay']])
flights_df.head()

,FlightDate,IATA_Code_Marketing_Airline,Operated_or_Branded_Code_Share_Partners,Operating_Airline,Flight_Number_Marketing_Airline,Origin,Dest,DepDelay,DepDelayMinutes,DepDel15,ArrDelay,ArrDelayMinutes,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2023-01-22,B6,B6,B6,1447,HPN,TPA,-4.0,0.0,0.0,-4.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,2023-01-22,B6,B6,B6,1451,BOS,MCO,53.0,53.0,1.0,54.0,54.0,0.0,0.0,39.0,0.0,1.0,0.0,14.0
2,2023-01-22,B6,B6,B6,1453,FLL,SJU,7.0,7.0,0.0,23.0,23.0,0.0,0.0,7.0,0.0,16.0,0.0,0.0
3,2023-01-22,B6,B6,B6,1454,SJU,FLL,7.0,7.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,2023-01-22,B6,B6,B6,1455,BOS,DCA,-8.0,0.0,0.0,-17.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [5]:
# Empty and delete the extraction_path directory
try:
    shutil.rmtree(extraction_path)
    print(f"Directory {extraction_path} and all its contents have been deleted.")
except OSError as e:
    print(f"Error: {e.strerror}")

Directory Resources/BTS/extracted_zips and all its contents have been deleted.


## Explore and filter flights and airports to matching data

In [6]:
# number of unique origin airports in data
origins_df = unfiltered_flight_info[['Origin','OriginCityName']]
origins_df['Origin'].nunique()

359

In [7]:
# number of unique destination airports in data
destinations_df = unfiltered_flight_info[['Dest','DestCityName']]
destinations_df['Dest'].nunique()

359

In [8]:
# total number of unique airports in data
# compare the origin airports to destination airports
ori_match = origins_df['Origin'].isin(destinations_df['Dest'])
dest_match = destinations_df['Dest'].isin(origins_df['Origin'])
ori_dest_match = ori_match | dest_match

# only keep rows for airports that are listed in both columns
flight_airports = flights_df[ori_dest_match]
flight_airports['Origin'].nunique()
# confirmed that the same 359 airports have travel data

359

In [9]:
# Read in CSV of cleaned us-airports and compare to airports in flight data
us_airports = pd.read_csv('Resources/us-airports-cleaned.csv')
iata_match = us_airports['iata_code'].isin(flight_airports['Origin'])

us_airports_filtered = us_airports[iata_match].reset_index()
us_airports_filtered['iata_code'].nunique()

351

In [10]:
# Which 8 airports do we have data for that our airport list doesn't have?
iata_match = flight_airports['Origin'].isin(us_airports_filtered['iata_code'])

us_airports_missing = flight_airports[~iata_match]
us_airports_missing['Origin'].unique()

array(['SJU', 'STT', 'BQN', 'PSE', 'STX', 'PPG', 'GUM', 'SPN'],
      dtype=object)

Research (Google):
SJU, BQM, PSE = Puerto Rico
STT,STX, = U.S. Virgin Islands
PPG = American Samoa (unincorporated territory)
GUM = Guam
SPN = Northern Mariana Islands

In [11]:
# Excluding data from the 8 erroneous airports 
exclusions = flights_df['Origin'].isin(us_airports_missing['Origin'])

filtered_flights = flights_df[~exclusions]
filtered_flights['Origin'].nunique()

351

In [12]:
# Filter airports to just those that have flight data
airport_mask = us_airports['iata_code'].isin(filtered_flights['Origin'])

us_airports_filtered = pd.DataFrame(us_airports[airport_mask])
us_airports_filtered['iata_code'].nunique()

351

In [13]:
us_airports_filtered['type'].value_counts()

type
medium_airport    276
large_airport      65
small_airport      10
Name: count, dtype: int64

In [14]:
# Export filtered data sets to CSV
us_airports_filtered.to_csv('Resources/us_airports_filtered.csv')
filtered_flights.to_csv('Resources/filtered_flights.csv')

In [15]:
# # Zip filtered_flights.csv and delete to allow github upload
csv_path = 'Resources/filtered_flights.csv'
zip_path = 'Resources/filtered_flights.zip'

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_path, os.path.basename(csv_path))

# Check if the ZIP file exists and the size condition is met
if os.path.exists(zip_path):
    # Get the size of the ZIP file in bytes
    zip_size = os.path.getsize(zip_path)
    
    # Define the size condition, e.g., file must be larger than 0 bytes to ensure it's not empty
    if zip_size > 50:
        os.remove(csv_path)
        print(f"The file {csv_path} has been zipped, is not empty, and has been deleted.")
    else:
        print(f"The file {zip_path} is empty.")
else:
    print("Error: The zip file was not created successfully.")

The file Resources/filtered_flights.csv has been zipped, is not empty, and has been deleted.


## Identify Airlines

In [39]:
# Identify unique IATA codes and store in a list in order of frequency
airline_IATA_codes = filtered_flights['IATA_Code_Marketing_Airline'].value_counts().index.to_list()
airline_IATA_codes

['AA', 'DL', 'WN', 'UA', 'AS', 'B6', 'NK', 'F9', 'G4', 'HA']

In [42]:
# Identify airlines and make list in matching order 
# Source: https://www.iata.org/en/publications/directories/code-search/
airline_names = ['American Airlines','Delta Airlines','Southwest Airlines','United Airlines',
                 'Alaska Airlines','JetBlue','Spirit Airlines','Frontier Airlines','Allegiant Air',
                 'Hawaiian Airlines']
# Build dictionary of IATA_codes and airline names to turn into a data frame
airline_data = {'airline_IATA_code':airline_IATA_codes,
               'airline_name':airline_names}

,airline_IATA_code,airline_name
0,AA,American Airlines
1,DL,Delta Airlines
2,WN,Southwest Airlines
3,UA,United Airlines
4,AS,Alaska Airlines
5,B6,JetBlue
6,NK,Spirit Airlines
7,F9,Frontier Airlines
8,G4,Allegiant Air
9,HA,Hawaiian Airlines


In [43]:
# create dataframe
airline_df = pd.DataFrame(airline_data)
airline_df

,airline_IATA_code,airline_name
0,AA,American Airlines
1,DL,Delta Airlines
2,WN,Southwest Airlines
3,UA,United Airlines
4,AS,Alaska Airlines
5,B6,JetBlue
6,NK,Spirit Airlines
7,F9,Frontier Airlines
8,G4,Allegiant Air
9,HA,Hawaiian Airlines


In [44]:
# Export to CSV
airline_df.to_csv('Resources/airlines.csv')

In [ ]:
## Get city of airport from the flight data and insert into airport information for SQL